<a href="https://colab.research.google.com/github/Niirav213/Early-warning-for-critical-events-Predict-things-like-sepsis-or-heart-failure-before-they-happen/blob/main/sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# =========================
# BUILD PREPROCESSING PIPELINE
# =========================
def build_pipeline(df, target="SepsisLabel", missing_threshold=0.95):
    """
    Build and fit preprocessing pipeline on a GLOBAL dataframe.
    """

    # 1. Drop highly missing features (GLOBAL missingness)
    missing_frac = df.isnull().mean()
    features_to_drop = missing_frac[missing_frac > missing_threshold].index.tolist()
    df_reduced = df.drop(columns=features_to_drop)

    # 2. Separate numeric and categorical features
    num_features = (
        df_reduced
        .select_dtypes(include=["float64", "int64"])
        .drop(columns=[target])
        .columns
        .tolist()
    )

    cat_features = ["Gender", "Unit1", "Unit2"]
    cat_features = [c for c in cat_features if c in df_reduced.columns]
    num_features = [c for c in num_features if c not in cat_features]

    # 3. Transformers
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    # 4. ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_features),
            ("cat", categorical_transformer, cat_features)
        ]
    )

    # 5. Final pipeline
    pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

    # Fit pipeline on GLOBAL data
    X = df_reduced.drop(columns=[target])
    pipeline.fit(X)

    return pipeline


# =========================
# APPLY PIPELINE
# =========================
def apply_pipeline(df, pipeline, target="SepsisLabel"):
    X = df.drop(columns=[target])
    y = df[target]

    X_preprocessed = pipeline.transform(X)
    feature_names = pipeline.named_steps["preprocessor"].get_feature_names_out()

    X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=feature_names)

    return X_preprocessed_df, y


# =========================
# LOAD GLOBAL SAMPLE (KEY FIX)
# =========================
def load_global_sample(input_folder, max_files=200):
    """
    Load data from multiple patient files to estimate GLOBAL missingness.
    Prevents dropping labs absent in a single patient.
    """
    dfs = []
    count = 0

    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.endswith(".psv"):
                df = pd.read_csv(os.path.join(root, f), sep="|")
                dfs.append(df)
                count += 1

                if count >= max_files:
                    return pd.concat(dfs, ignore_index=True)

    return pd.concat(dfs, ignore_index=True)


# =========================
# PROCESS DATASET FOLDER
# =========================
def main(input_folder, output_folder, target="SepsisLabel"):
    os.makedirs(output_folder, exist_ok=True)

    # Find all PSV files
    psv_files = []
    for root, _, files in os.walk(input_folder):
        for f in files:
            if f.endswith(".psv"):
                psv_files.append(os.path.join(root, f))

    if not psv_files:
        print(f"No PSV files found in {input_folder}")
        return

    print(f"Found {len(psv_files)} PSV files in {input_folder}")

    # 🔑 GLOBAL PIPELINE FIT (FIX)
    print("Building global sample for preprocessing...")
    global_df = load_global_sample(input_folder, max_files=200)
    print(f"Global sample shape: {global_df.shape}")

    pipeline = build_pipeline(global_df, target=target)

    # Process each patient file
    for file_path in psv_files:
        filename = os.path.basename(file_path)
        print(f"Processing {filename}...")

        df = pd.read_csv(file_path, sep="|")
        X_preprocessed, y = apply_pipeline(df, pipeline, target=target)

        df_out = pd.concat(
            [X_preprocessed, y.reset_index(drop=True)],
            axis=1
        )

        output_path = os.path.join(output_folder, filename)
        df_out.to_csv(output_path, sep="|", index=False)

    print(f"✅ Finished processing {input_folder}")


# =========================
# ENTRY POINT
# =========================
if __name__ == "__main__":

    # TRAINING (SET A)
    print("Processing training_setA...")
    main(
        input_folder="training",
        output_folder="processed_training_Ay"
    )

    # VALIDATION / TEST (SET B)
    print("Processing training_setB...")
    main(
        input_folder="training_setB",
        output_folder="processed_training_setB_Ay"
    )


Processing training_setA...
Found 20336 PSV files in training
Building global sample for preprocessing...
Global sample shape: (7556, 41)
Processing p000001.psv...
Processing p000002.psv...
Processing p000003.psv...
Processing p000004.psv...
Processing p000005.psv...
Processing p000006.psv...
Processing p000007.psv...
Processing p000008.psv...
Processing p000009.psv...
Processing p000010.psv...
Processing p000011.psv...
Processing p000012.psv...
Processing p000013.psv...
Processing p000014.psv...
Processing p000015.psv...
Processing p000016.psv...
Processing p000017.psv...
Processing p000018.psv...
Processing p000019.psv...
Processing p000020.psv...
Processing p000021.psv...
Processing p000022.psv...
Processing p000023.psv...
Processing p000024.psv...
Processing p000025.psv...
Processing p000026.psv...
Processing p000027.psv...
Processing p000028.psv...
Processing p000029.psv...
Processing p000030.psv...
Processing p000031.psv...
Processing p000032.psv...
Processing p000033.psv...
Proc

X_train shape: (19900, 336, 32)
X_val shape: (19658, 336, 32)
TIMESTEPS: 336
N_FEATURES: 32


{0: np.float64(0.536503828318775), 1: np.float64(7.348596750369277)}


Training with: {'dropout': 0.3, 'units': 32}
Epoch 1/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 72s 110ms/step - auc: 0.7561 - loss: 0.5602 - val_auc: 0.7982 - val_loss: 0.6667
Epoch 2/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 78s 104ms/step - auc: 0.9037 - loss: 0.3787 - val_auc: 0.7942 - val_loss: 0.6792
Epoch 3/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 90s 117ms/step - auc: 0.9275 - loss: 0.3383 - val_auc: 0.7981 - val_loss: 0.7247
Epoch 4/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 64s 103ms/step - auc: 0.9230 - loss: 0.3338 - val_auc: 0.7784 - val_loss: 0.9467
Epoch 5/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - auc: 0.9424 - loss: 0.3074 - val_auc: 0.7800 - val_loss: 0.8731
Epoch 6/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 89s 114ms/step - auc: 0.9494 - loss: 0.2764 - val_auc: 0.7994 - val_loss: 0.7240
Epoch 7/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 67s 108ms/step - auc: 0.9561 - loss: 0.2630 - val_auc: 0.7870 - val_loss: 0.9024
Epoch 8/10
622/622 ━━━━━━━━━━━━━━━━━━━━ 64s 103ms/step - auc: 0.9556 - loss: 0.2674 - val_auc: 0.7989 - val_

NameError: name 'scale_pos_weight' is not defined